# Lab Sheet 8 (COM3502-4502-6502 Speech Processing)

This lab sheet is part of the lecture COM[3502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level3/com3502.html "Open web page for COM3502 module")-[4502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level4/com4502.html "Open web page for COM4502 module")-[6502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/msc/com6502.html "Open web page for COM4502 module") Speech Processing at the [University of Sheffield](https://www.sheffield.ac.uk/ "Open web page of The University of Sheffield"), Dept. of [Computer Science](https://www.sheffield.ac.uk/dcs "Open web page of Department of Computer Science, University of Sheffield").

It is probably easiest to open this Jupyter Notebook with [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true "Open in Google Colab") since GitHub's Viewer does not always show all details correctly. <a href="https://colab.research.google.com/github/sap-shef/SpeechProcesssingLab/blob/main/Lab-Sheets/Lab-Sheet-8.ipynb"><img align="right" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open Notebook in Google Colab" title="Open and Execute the Notebook directly in Google Colaboratory"></a>

Please put questions, comments and correction suggestions in the [Blackboard](https://vle.shef.ac.uk) discussion board or send an email to [s.goetze@sheffield.ac.uk](mailto:s.goetze@sheffield.ac.uk).

<div class="alert alert-block alert-success" id='ILOs'>
<strong>Intended Learning Objectives (ILOs):</strong><br>
    
After completing this Jupyter Notebook you should
    
<ul>
<li>Understand the concept of <a href="https://en.wikipedia.org/wiki/Overlap%E2%80%93add_method"><code>Overlap-add algorithm</code></a>
</li> 
<li>Be able to implement your own Overlap-add function
</li>    
<li>Understand the concept of <a href="https://wiki.aalto.fi/display/ITSP/Windowing"><code>Perfect Reconstruction</code></a> and the Princen-Bradley criteria
</li>
<li>Denoise a modelled microphone signal via Overlap-add Wiener filtering
</li>
</ul>
</div>

In [3]:
# Let's do the ususal necessary and nice-to-have imports
%matplotlib inline
import matplotlib.pyplot as plt  # plotting
import seaborn as sns; sns.set() # styling ((un-)comment if you want)
import numpy as np               # math

# imports we need in addition for this lab sheet
from IPython import display as ipd
import scipy.signal as sig
import soundfile as sf
from scipy.io.wavfile import read as wavread

<br>
<a id='task_1'></a>
<div class="alert alert-block alert-info">
    
**Task 1:**
    
<ul>
<li> 
    Load a WAVE file containing speech, e.g. <code>speech_8kHz_murder.wav</code> from the Internet address <code>https://staffwww.dcs.shef.ac.uk/people/S.Goetze/sound/</code> and load it into a variable <code>s</code>.
</li>
    <li> 
    Generate a signal of white noise of the same length as your speech signal and load it into a variable <code>n</code>. Refer to lab sheet 6 for help.
</li>
  <li>   
    Create a microphone signal $y[k] = s[k]+n[k]$ as shown in the schematic below. Note that the vectors should be cut the same length if needed and the magnitude of the noise signal shouldn't completely drown out the speech signal. Also note that we will do the filtering in the following. 

<img src="NRSingleChannelBasic-web.png" align="center"/>
    </li> 
</ul>
</div>

In [2]:
# your code here:
# load speech wave into variable, s
#...

# generate white noise signal in variable, n
#...

# combine both signals to create your microphone signal, y
#...

# listen to your signal (if you want)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  328k  100  328k    0     0  1721k      0 --:--:-- --:--:-- --:--:-- 1721k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57808  100 57808    0     0   482k      0 --:--:-- --:--:-- --:--:--  482k


# Overlap-add Processing

When processing long duration signals it is common practise to do so by splitting the long signal into a series of shorter segments or chunks. This is primarily for three reasons. First, dealing with the full signal 'in one go' might be impractical due to constraints in computational memory. Second, to attain useful inference from operations such as the discrete Fourier Transform, it is assumed that the signal under scrutiny is time invariant (i.e its statistical properties do not vary with time), this is only typically true when analysing signals over short time frames. Third, for streaming applications where you may be interested in the result whilst still obtaining input data, such as adaptive noise reduction for a phone call.

A simple algorithm to perform such chunk-wise processing is the Overlap-Add method.

Overlap-Add processing can effectively be broken down into 3 stages:
<ul>
<li> 
    <strong>Fragmentation</strong> - Extract windows of size $L$ by iteratively applying a windowing function, such as a Hanning window, to each chunk with each chunk overlapping by a number of $M$ samples. $M$ is often chosen to be $L/2$ in speech processing, i.e the overlapping fraction is that of the window length halved.
</li> 
<li>
    <strong>Processing</strong> - Each windowed chunk is processed independently, by convolving with a FIR filter for example.
</li>
<li>
    <strong>Reconstruction</strong> - The processed chunks are 'added' back together sequentially to retain each chunk's relative position in the original signal.
</li>
</ul>
    

The process is described mathematically below where a long signal, $x[k]$, is to be convolved with a finite impulse response filter, $h[k]$.  

1.&emsp;Applying a window function centered at the $i$th sample with window $w$ of length $L$, whose value is 0 beyond $-L/2$ and $+L/2$:

$$x_i[k] = x[k]w^L_i[k]$$

2.&emsp;Fragmentation of signal:

$$x[k] = x_0[k] + x_1[k] + x_2[k] + ... $$

3.&emsp;Application of filter to fragmented signal: 

$$y[k]=\sum_{i=0}^{N-1}x_{k-i}\cdot h[i]$$  

4.&emsp;Use the distributive property of convolution to formulate as blockwise:
\begin{align*}
y[k]=&\sum_{i=0}^{N-1}x[k-i]h[i]
\\=&\sum_{i=0}^{N-1}(x_{0}[k-i]+x_{1}[k-i]+...)\cdot h[i]
\\=&\sum_{i=0}^{N-1}x_{0}[k-i]\cdot h[i]+\sum_{i=0}^{N-1}x_{1}[k-i]\cdot h[i] + ...
\\=&\:\mathrm{IFFT}\left \{ x_{0}[n]h[n]\right \} + \mathrm{IFFT}\left \{ x_{1}[n]h[n]\right \} +...
\\=&\: y_{0}[k] + y_{1}[k] + ...
\end{align*}

Since speech signals are time variant in nature, it is beneficial to utilise the Overlap-Add algorithm during the processing of them. This is based on the assumption that, heuristically speaking, fragmented speech present in each chunk may roughly represents a phoneme.



<br>
<a id='task_1'></a>
<div class="alert alert-block alert-info">
    
**Task 2: Perform windowing on your modelled microphone signal.**
    
<ul>
<li> Extract a short temporal range from your modelled microphone signal; around 1 second of audio.
</li>     
<li>
    Finish the below function to iterate through said range applying a windowing function from the numpy library to each iteration. 
</li>
<li>
       Have the function return a list or np.array() all extracted chunks, with each chunk padded with zeros so that all chunks are of the same length as the original input.
</li>
<li>
       Test your function with the Blackman filter.
</li>
</ul>
</div>

In [1]:
def get_windows(sample, chunk_length = 0.03, fs = 8000, window_choice = 'Blackman'):
    """
    Extracts windowed segments from a signal for a given phoneme duration using a windowing
    function from the numpy library.
    
    Input:
        sample: list or np.array
            input signal to window
        chunk_length: float
            desired chunk length in seconds
        fs: int
            sampling frequency of signal
        window_choice: str
            windowing function to use, one of: Blackman, Hamming, Hanning
    Output:
        list of np.arrays
    
    Example:
       chunks = get_windows(sample, chunk_length = 0.025, fs=8000, window_choice = 'hanning')
    """
    ## suggested method: 
    # find chunk centres first
    # iterate through your window centres with a for loop
    # use a dictionary to accept the window choice argument as an input and output the appropriate function from np
    
    return chunks

## Princen-Bradley Criteria

A desired property of the windowing process used in an Overlap-Add algorithm is that the windowing itself should not significantly alter the original signal after signal reconstruction. This is known as the Princen-Bradley Criteria.


<br>
<a id='task_1'></a>
<div class="alert alert-block alert-info">
    
**Task 3: Try your windowing function with the phoneme_duartion as 0.03 seconds and window_choice = 'Blackman'. Do you think the output from this configuration satisfies the Princeton-Bradley Criteria?**
    
**Hint**: You can test this by simply summing the output of your window function and subtracting this from your original shortened microphone signal.
</div>

In [ ]:
# your code here
# reconstruct your windowed signal
#...

# plot
#...

## Weiner Filtering
A simple method to reduce the effects of the white noise we added to our modelled microphone signal in task 1 is to apply a Wiener filter. The Wiener filter is defined below where $h[n]$ is the impulse response of the filter $\Phi_{yy}$, is the Power Spectral Density of the filter and $\Phi_{n}$ is the Power Spectral Density of the filter.

$$h[n]=\frac{\Phi_{yy}-\Phi_{n}}{\Phi_{yy}}$$

<br>
<a id='task_1'></a>
<div class="alert alert-block alert-info">

**Task 4: Apply the Wiener filter to 2 chunks of your modelled microphone signal**
    
<ul>
<li> Use scipy.sig.wiener() to apply Wiener filtering to each of the 2 chunks from your modelled microphone signal.
</li>  
<li> Plot the power spectral density function before and after applying the filter for each chunk.
</li>     
</ul>
</div>


In [1]:
#your code here..


## Overlap-Add Function
Now that we have observed the effects of the Wiener filter and how windowing can be performed, you should now have the tools to create your own Overlapp-add Weiner filtering function. 

<br>
<a id='task_1'></a>
<div class="alert alert-block alert-info">
    
    
**Task 5: Finish the function below to implement the Overlap-Add algorithm to perform Weiner filtering.**
    
<ul>
<li> Create a list of evenly spaced chunks from your signal, with a 50% overlapping fraction.
</li>     
<li> Apply a Hanning window to each chunk.
</li>     
<li> 
   Perform Wiener filtering using the scipy library on each chunk.
</li> 
<li> 
   Reconstruct each filtered chunk to obtain the full filtered signal.
</li>
<li>
     Aurally compare with the non-filtered signal, have you successfully denoised the signal?
</li>
</ul>
</div>

In [2]:
def overlap_add_filtering(signal, chunk_length):
    """
    Overlap-Add filtering.
    
    Input:
        signal: np.array or list
            input signal to process
        chunk_length: int
            size of 
    Output:
        filtered signal
    
    Example:
       overlap_add_filtering(signal, testfilt, 1000)
    """
    bits_to_shift_by = (L_I-1).bit_length() # number of bits necessary to represent in binary
    L_F = 2<<bits_to_shift_by # length of fourier transform window
    L_W = L_F - L_I + 1 # length of signal to be filtered in single window overlap iteration
    
    filt_F = np.fft.rfft(filt, n=L_F) # filter in frequency domain
    
    
    #YOUR WORK HERE
    
    
    
        
    return signal_f[:len(signal)] #truncate to length of original signal

In [ ]:
# call to test your function
# ...

# aurally compare with noisy signal

## Copyright

This notebook is licensed to be used during the lecture COM[3502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level3/com3502.html "Open web page for COM3502 module")-[4502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level4/com4502.html "Open web page for COM4502 module")-[6502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/msc/com6502.html "Open web page for COM4502 module") Speech Processing at the [University of Sheffield](https://www.sheffield.ac.uk/ "Open web page of The University of Sheffield"), Dept. of [Computer Science](https://www.sheffield.ac.uk/dcs "Open web page of Department of Computer Science, University of Sheffield"). Any further use (beyond use for the lecture) is only permitted if agreed with the [module lead](mailto:s.goetze@sheffield.ac.uk). 